# Verifying zeros of the Lorenz system with Interval Arithmetic

We begin by installing the interval arithmetic package and loading the other packages that will be used for the computations.

In [24]:
using Pkg

In [25]:
# Install the IntervalArithmetic package
# Just need to do this once
Pkg.add("IntervalArithmetic")

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


In [26]:
using Polynomials
using LinearAlgebra
using IntervalArithmetic

Now we define the vector field $f$ and its derivative $Df$ for the Lorenz equations, which are functions of a variable $x \in \mathbb{R}^3$ and three parameters $\sigma$, $\rho$, and $\beta$ which we denote by $s$, $r$, and $b$ respectively.

In [27]:
f(x, s, r, b) = [s * (x[2] - x[1]), r * x[1] - x[2] - x[1] * x[3], - b * x[3] + x[1] * x[2]]

f (generic function with 1 method)

In [28]:
Df(x, s, r, b) = [-s s 0; r - x[3] -1 -x[1]; x[2] x[1] -b]

Df (generic function with 1 method)

We define the parameters as intervals containing the parameter values of interest.

In [29]:
si = @interval(10); ri = @interval(28); bi = @interval(8/3);

For the numerical (non-rigorous) computations we do not want to use intervals, so we define floating point versions of the parameter values as the upper limits of the interval parameters.

In [30]:
# We could also use the lower limits
# s = si.lo; r = ri.lo; b = bi.lo;
s = si.hi; r = ri.hi; b = bi.hi;

Next we define the approximate solution $\bar{x}$.

In [31]:
x_bar = [8.4853, 8.4853, 27];

The first step in applying the radii polynomial theorem is to choose $A$. We do not want to compute the inverse using interval arithmetic since it is expensive. So we compute the inverse numerically using floating point arithmetic.

In [32]:
A = inv(Df(x_bar, s, r, b))

3×3 Array{Float64,2}:
 -0.0518518  -0.0185184  0.0589254
  0.0481482  -0.0185184  0.0589254
 -0.0117851  -0.117851   0.0      

Then we define interval versions of $A$ and $\bar{x}$ to be used for the rigorous computations of the bounds.

In [33]:
Ai = interval.(A)

3×3 Array{Interval{Float64},2}:
 [-0.0518519, -0.0518518]  …      [0.0589254, 0.0589255]
  [0.0481481, 0.0481482]          [0.0589254, 0.0589255]
 [-0.0117851, -0.011785]      [0, 0]                    

In [34]:
x_bar_i = interval.(x_bar)

3-element Array{Interval{Float64},1}:
       [8.4853, 8.48531]
       [8.4853, 8.48531]
 [27, 27]               

Now define the rigorous bounds, using interval arithmetic, for the radii polynomial theorem.

In [35]:
Y0i = norm(Ai * f(x_bar_i, si, ri, bi), Inf)

[1.86257e-05, 1.86258e-05]

Notice that we do not need to define the identity matrix, $I$, below. Julia uses the symbols $\textsf{I}$ to represent the identity matrix and it infers its size from the expression where it is used.

In [36]:
Z0i = norm(I - Ai * Df(x_bar_i, si, ri, bi), Inf)

[0, 1.11023e-16]

In [37]:
Z2i = 2 * max(abs(Ai[1,2]) + abs(Ai[1,3]), abs(Ai[2,2]) + abs(Ai[2,3]), abs(Ai[3,2]) + abs(Ai[3,3]))

[0.235701, 0.235702]

To find the radius $r_0$ in the radii polynomial theorem we do not need to use interval arithemetic, hence we define floating point values for the coefficients of the polynomial and find its roots numerically. We use the upper limits of the intervals as the bounds we need. Notice that the values $Y_0$, $Z_0$, and $Z_2$ computed below are rigorous upper bounds since they are the upper limits of the intervals computed rigorously with interval arithmetic.

In [38]:
Y0 = Y0i.hi; Z0 = Z0i.hi; Z2 = Z2i.hi;

The radii polynomial is given by $p(r) = Z_2 r^2 - (1 - Z_0) r + Y_0$.

In [39]:
p = Poly([Y0, -(1 - Z0), Z2])

Poly(1.862574098998336e-5 - 0.9999999999999999*x + 0.23570174301438956*x^2)

Finally we find the roots of the radii polynomial.

In [40]:
rs = roots(p)

2-element Array{Float64,1}:
 4.24263137417724     
 1.8625822759932215e-5

We call the roots $r_1$ and $r_2$, and define $r_0$ as the mid point of interval $[r_1, r_2]$. This will give us one radius $r_0$ of existence by the radii polynomial theorem. Notice that we could also define an interval $[r_{-}, r_{+}]$ slightly smaller that $[r_1, r_2]$ and use the radii polynomial theorem to get an interval, $[r_{-}, r_{+}]$, of radii of existence and uniqueness.

In [41]:
r1 = rs[2]; r2 = rs[1]; r0 = (r1 + r2) / 2.0;

Now we define an interval version of $r_0$ and compute $p(r_0)$ using interval arithemetic to rigorously check that $p(r_0) < 0$.

In [42]:
r0i = @interval(r0)

[2.12132, 2.12133]

In [43]:
p_r0i = Z2i * r0i^2 -(1 - Z0i) * r0i + Y0i

[-1.06065, -1.06064]

Finally all that is left to do is to check if $p(r_0) < 0$. This check is rigorous, as it is done using interval arithmetic.

In [44]:
if p_r0i < 0
  println("Rigorous Proof Successful!")
else
  println("Rigorous Proof Failed!")
end

Rigorous Proof Successful!
